In [1]:
import os
import glob
import math
import re

# from mpmath import *
# from config import *
# from osgeo import gdal

## Conversion of tile to lat and lon in degrees
## This function takes in x, y, and zoom variables and outputs lat and lon in degrees

###### Link to resource: - https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Tile_numbers_to_lon./lat._2

In [2]:
def tiletolatlon(xtile, ytile, zoom):
    """
    conversion of tile values to lon and lat in degrees"""
    n = 2.0**zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = lat_rad * 180.0 / math.pi
    return lon_deg, lat_deg

## Conversion of lat-lon to EPSG 3857
##### Link to resource: - https://gis.stackexchange.com/questions/142866/converting-latitude-longitude-epsg4326-into-epsg3857

In [3]:
def ConvertCoor(lon, lat):

    """
    Converts lat lon  in degrees to epsg 3857"""

    lonInEPSG3857 = lon * 20037508.34 / 180
    latInEPSG3857 = (
        math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    ) * (20037508.34 / 180)

    # print("{0},{1}".format(lonInEPSG3857,latInEPSG3857))
    return lonInEPSG3857, latInEPSG3857

## Clipping of tiles to GeoJSON file with the following inputs: - 

- minimum value of x
- minimum value of y
- maximum value of x
- maximum value of y

### Function sends command to clip to command line

In [4]:
def clipping(xmin, ymin, xmax, ymax, fname, n):
    clip = f"ogr2ogr -clipsrc {xmin} {ymin} {xmax} {ymax} -f GeoJSON {fname}.geojson {n}/labels_rep.geojson"
    # os.system(clip)
    #! clip
    print(clip)
    return clip

# Rasterizing of georeferenced tiles file with the following inputs: - 

- minimum value of x
- minimum value of y
- maximum value of x
- maximum value of y
- filename

### Function sends command to clip to command line to rasterize image with provided parameters

In [5]:
def raster(xmin, ymin, xmax, ymax, fname):
    raster = f"gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te {xmin} {ymin} {xmax} {ymax} {fname}.geojson\
    {fname}.tiff"
    os.system(raster)
    print(raster)
    return raster

## Reproject GeoJSON files from EPSG:4326 to EPSG:3857

In [6]:
def label_converter(n):
    conv = f"ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 -f GeoJSON {n}/labels_rep.geojson {n}/labels.geojson"
    os.system(conv)
    # print(conv)
    return conv

## Conversion of rasterized tiff file to png with the following inputs: - 

- filename

### Function sends command to convert using gdal_translate to command line

In [7]:
def tifftopng(fname):
    conv = f"gdal_translate -ot Byte -of PNG -scale -co worldfile=yes {fname}.tiff {fname}-burned.png"
    print(conv)
    return conv

## Below is to run the process of georeferencing and rasterization for every individual tile image in every of the five available folders iteratively with the following steps: -

- iterate through individual folders
- iterate through each image in the current folder
   - georefernce images
   - clip images
   - rasterize images
   - convert from tiff to png

In [9]:
for i in range(5):

    path = f"{i+1}/*?.png"
    print("\033[1m" + f"Performing rasterizing on folder {i+1}" + "\033[0m")

    # labels converter
    cmd = label_converter(
        i + 1
    )  # start on compiling commands to georeference and rasterize tiles in entire folder to
    # be executed in batch files

    # iterate through current folder
    for tiles in glob.glob(path):

        fname = tiles.replace(".png", "")
        split = re.split("-", tiles)
        x = int(split[1])
        y = int(split[2])
        z = int(split[3].replace(".png", ""))
        print(x, y, z)

        name = list(fname)
        name[0] = f"{i+1}\After"
        name = "".join(name)
        print(name)

        # get minimum and maximum lat and lon in degrees
        lon_min, lat_min = tiletolatlon(x, y + 1, z)
        lon_max, lat_max = tiletolatlon(x + 1, y, z)
        print(lon_min, lat_min)
        print(lon_max, lat_max)

        # conversion to espg3857
        x_min, y_min = ConvertCoor(lon_min, lat_min)
        x_max, y_max = ConvertCoor(lon_max, lat_max)

        # the following lines can be uncomented to view bounding box
        # print(x_min, y_min)
        # print(x_max, y_max)

        # clipping of tile into geojson
        clip = clipping(x_min, y_min, x_max, y_max, name, i + 1)
        cmd += "&" + clip

        # generate raster of tile image
        ras = raster(x_min, y_min, x_max, y_max, name)
        cmd += "&" + ras

        # convert tiff to png
        co = tifftopng(name)
        cmd += "&" + co

        print()

    print("\033[1m" + f"Commands for rasterizing on folder {i+1}" + "\033[0m")
    print(cmd)  # complete command to georeference and rasterize entire folders
    # saves joint commands into batch files that can be executed
    with open(f"{i+1}commands.bat", "w") as f:
        f.write(cmd)

    print()
    print()

Performing rasterizing on folder 1
1277168 1083840 21
1\After\OAM-1277168-1083840-21
39.24041748046875 -6.0424068193735545
39.2405891418457 -6.042236111676103
ogr2ogr -clipsrc 4368223.291833191 -673887.95052538 4368242.40109026 -673868.8412683101 -f GeoJSON 1\After\OAM-1277168-1083840-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368223.291833191 -673887.95052538 4368242.40109026 -673868.8412683101 1\After\OAM-1277168-1083840-21.geojson    1\After\OAM-1277168-1083840-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277168-1083840-21.tiff 1\After\OAM-1277168-1083840-21-burned.png

1277168 1083841 21
1\After\OAM-1277168-1083841-21
39.24041748046875 -6.042577527017167
39.2405891418457 -6.0424068193735545
ogr2ogr -clipsrc 4368223.291833191 -673907.0597824476 4368242.40109026 -673887.95052538 -f GeoJSON 1\After\OAM-1277168-1083841-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368242.40109026 -673945.278296585 4368261.510347328 -673926.169039517 1\After\OAM-1277169-1083843-21.geojson    1\After\OAM-1277169-1083843-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277169-1083843-21.tiff 1\After\OAM-1277169-1083843-21-burned.png

1277169 1083844 21
1\After\OAM-1277169-1083844-21
39.2405891418457 -6.043089649624966
39.240760803222656 -6.042918942142874
ogr2ogr -clipsrc 4368242.40109026 -673964.3875536526 4368261.510347328 -673945.278296585 -f GeoJSON 1\After\OAM-1277169-1083844-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368242.40109026 -673964.3875536526 4368261.510347328 -673945.278296585 1\After\OAM-1277169-1083844-21.geojson    1\After\OAM-1277169-1083844-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277169-1083844-21.tiff 1\After\OAM-1277169-1083844-21-burned.png

1277169 1083845 21
1\Af

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368261.510347328 -673945.278296585 4368280.619604397 -673926.169039517 1\After\OAM-1277170-1083843-21.geojson    1\After\OAM-1277170-1083843-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277170-1083843-21.tiff 1\After\OAM-1277170-1083843-21-burned.png

1277170 1083844 21
1\After\OAM-1277170-1083844-21
39.240760803222656 -6.043089649624966
39.24093246459961 -6.042918942142874
ogr2ogr -clipsrc 4368261.510347328 -673964.3875536526 4368280.619604397 -673945.278296585 -f GeoJSON 1\After\OAM-1277170-1083844-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368261.510347328 -673964.3875536526 4368280.619604397 -673945.278296585 1\After\OAM-1277170-1083844-21.geojson    1\After\OAM-1277170-1083844-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277170-1083844-21.tiff 1\After\OAM-1277170-1083844-21-burned.png

1277170 1083845 21


gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368280.619604397 -673907.0597824476 4368299.728861466 -673887.95052538 1\After\OAM-1277171-1083841-21.geojson    1\After\OAM-1277171-1083841-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277171-1083841-21.tiff 1\After\OAM-1277171-1083841-21-burned.png

1277171 1083842 21
1\After\OAM-1277171-1083842-21
39.24093246459961 -6.04274823460694
39.24110412597656 -6.042577527017167
ogr2ogr -clipsrc 4368280.619604397 -673926.169039517 4368299.728861466 -673907.0597824476 -f GeoJSON 1\After\OAM-1277171-1083842-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368280.619604397 -673926.169039517 4368299.728861466 -673907.0597824476 1\After\OAM-1277171-1083842-21.geojson    1\After\OAM-1277171-1083842-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277171-1083842-21.tiff 1\After\OAM-1277171-1083842-21-burned.png

1277171 1083843 21
1\

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368280.619604397 -674174.5893814085 4368299.728861466 -674155.4801243402 1\After\OAM-1277171-1083855-21.geojson    1\After\OAM-1277171-1083855-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277171-1083855-21.tiff 1\After\OAM-1277171-1083855-21-burned.png

1277172 1083840 21
1\After\OAM-1277172-1083840-21
39.24110412597656 -6.0424068193735545
39.241275787353516 -6.042236111676103
ogr2ogr -clipsrc 4368299.728861466 -673887.95052538 4368318.838118534 -673868.8412683101 -f GeoJSON 1\After\OAM-1277172-1083840-21.geojson 1/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4368299.728861466 -673887.95052538 4368318.838118534 -673868.8412683101 1\After\OAM-1277172-1083840-21.geojson    1\After\OAM-1277172-1083840-21.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 1\After\OAM-1277172-1083840-21.tiff 1\After\OAM-1277172-1083840-21-burned.png

1277172 1083841 21

...

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4107534.8069028854 -145765.41291954071 4107573.0254170224 -145727.19440540377 5\After\OAM-631763-528101-20.geojson    5\After\OAM-631763-528101-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631763-528101-20.tiff 5\After\OAM-631763-528101-20-burned.png

631763 528102 20
5\After\OAM-631763-528102-20
36.89861297607422 -1.3096622448985584
36.898956298828125 -1.3093190118076414
ogr2ogr -clipsrc 4107534.8069028854 -145803.6314336776 4107573.0254170224 -145765.41291954071 -f GeoJSON 5\After\OAM-631763-528102-20.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4107534.8069028854 -145803.6314336776 4107573.0254170224 -145765.41291954071 5\After\OAM-631763-528102-20.geojson    5\After\OAM-631763-528102-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631763-528102-20.tiff 5\After\OAM-631763-528102-20-burned.png

631763 528103 20
5\After\

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108375.6142139053 -145459.66480644234 4108413.8327280427 -145421.44629230347 5\After\OAM-631785-528093-20.geojson    5\After\OAM-631785-528093-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631785-528093-20.tiff 5\After\OAM-631785-528093-20-burned.png

631785 528094 20
5\After\OAM-631785-528094-20
36.906166076660156 -1.3069163788560452
36.90650939941406 -1.3065731453895109
ogr2ogr -clipsrc 4108375.6142139053 -145497.88332057983 4108413.8327280427 -145459.66480644234 -f GeoJSON 5\After\OAM-631785-528094-20.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108375.6142139053 -145497.88332057983 4108413.8327280427 -145459.66480644234 5\After\OAM-631785-528094-20.geojson    5\After\OAM-631785-528094-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631785-528094-20.tiff 5\After\OAM-631785-528094-20-burned.png

631785 528095 20
5\Afte

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108452.05124218 -145536.10183471662 4108490.269756317 -145497.88332057983 5\After\OAM-631787-528095-20.geojson    5\After\OAM-631787-528095-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631787-528095-20.tiff 5\After\OAM-631787-528095-20-burned.png

631788 528088 20
5\After\OAM-631788-528088-20
36.907196044921875 -1.3048569773536338
36.90753936767578 -1.3045137436059036
ogr2ogr -clipsrc 4108490.269756317 -145268.57223575842 4108528.488270454 -145230.35372162 -f GeoJSON 5\After\OAM-631788-528088-20.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108490.269756317 -145268.57223575842 4108528.488270454 -145230.35372162 5\After\OAM-631788-528088-20.geojson    5\After\OAM-631788-528088-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631788-528088-20.tiff 5\After\OAM-631788-528088-20-burned.png

631788 528089 20
5\After\OAM-631788-

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108528.488270454 -145536.10183471662 4108566.7067845915 -145497.88332057983 5\After\OAM-631789-528095-20.geojson    5\After\OAM-631789-528095-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631789-528095-20.tiff 5\After\OAM-631789-528095-20-burned.png

631790 528088 20
5\After\OAM-631790-528088-20
36.90788269042969 -1.3048569773536338
36.908226013183594 -1.3045137436059036
ogr2ogr -clipsrc 4108566.7067845915 -145268.57223575842 4108604.925298729 -145230.35372162 -f GeoJSON 5\After\OAM-631790-528088-20.geojson 5/labels_rep.geojson
gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108566.7067845915 -145268.57223575842 4108604.925298729 -145230.35372162 5\After\OAM-631790-528088-20.geojson    5\After\OAM-631790-528088-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631790-528088-20.tiff 5\After\OAM-631790-528088-20-burned.png

631790 528089 20
5\After\OAM-631

gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4108604.925298729 -145536.10183471662 4108643.1438128664 -145497.88332057983 5\After\OAM-631791-528095-20.geojson    5\After\OAM-631791-528095-20.tiff
gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-631791-528095-20.tiff 5\After\OAM-631791-528095-20-burned.png

Commands for rasterizing on folder 5
ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:3857 -f GeoJSON 5/labels_rep.geojson 5/labels.geojson&ogr2ogr -clipsrc 4107267.277303925 -145708.0851483335 4107286.3865609933 -145688.97589126666 -f GeoJSON 5\After\OAM-1263512-1056200-21.geojson 5/labels_rep.geojson&gdal_rasterize -burn 255 -burn 255 -burn 255 -ts 256 256 -te 4107267.277303925 -145708.0851483335 4107286.3865609933 -145688.97589126666 5\After\OAM-1263512-1056200-21.geojson    5\After\OAM-1263512-1056200-21.tiff&gdal_translate -ot Byte -of PNG -scale -co worldfile=yes 5\After\OAM-1263512-1056200-21.tiff 5\After\OAM-1263512-1056200-21-burned.png&ogr2ogr -clips

In [50]:
print(len(cmd))

9132
